##  Preprocessing, Scenedetection ## 


In [ ]:

import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from scenedetect import open_video, SceneManager, ContentDetector
from IPython.display import display, clear_output
import time



# Function to safely get folder path from user
def get_folder_path(prompt, create_if_missing=False):
    while True:
        path = input(prompt).strip()
        if path:
            if os.path.exists(path) or create_if_missing:
                os.makedirs(path, exist_ok=True)  # Create folder if missing
                print(f"✅ Using folder: {path}")
                return path
        print("❌ Invalid or empty path. Please enter a valid folder path.")


input_folder = get_folder_path("Enter the input folder path where videos are stored (e.g., /Users/alicja/Desktop/VideoAnalyse/Corpus/Action): ")
output_folder = get_folder_path("Enter the output folder path for scene data (e.g., /Users/alicja/Desktop/VideoAnalyse/Corpus/SceneData/ActionData): ", create_if_missing=True)
output_video_folder = get_folder_path("Enter the output folder for split video segments (e.g., /Users/alicja/Desktop/VideoAnalyse/Corpus/VideoSegmentation/ActionSegments): ", create_if_missing=True)

# Function to detect scenes and save timestamps using PySceneDetect recommended approach
def detect_scenes(video_path, output_csv):
    print(f"📽️ Starting scene detection for {video_path}...")
    start_time = time.time()
    
    video = open_video(video_path)
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector(threshold=30))
    
    scene_manager.detect_scenes(video)
    scene_list = scene_manager.get_scene_list()
    
    scene_data = [(scene[0].get_seconds(), scene[1].get_seconds()) for scene in scene_list]
    scene_df = pd.DataFrame(scene_data, columns=["Start Time (s)", "End Time (s)"])
    scene_df["Video"] = os.path.basename(video_path).split('.')[0]
    scene_df.to_csv(output_csv, mode='a', index=False, header=not os.path.exists(output_csv))
    
    elapsed_time = time.time() - start_time
    print(f"✅ Completed scene detection for {video_path} in {elapsed_time:.2f} seconds.")
    print("Detected Scenes:")
    display(scene_df)
    
    return scene_df, [(float(scene[0].get_seconds()), float(scene[1].get_seconds())) for scene in scene_list]

# Function to split video based on detected scenes using FFmpeg
def split_video(video_path, scene_list):
    print(f"🎬 Splitting video {video_path} into scenes using FFmpeg...")
    video_filename = os.path.basename(video_path).split('.')[0]
    output_dir = os.path.join(output_video_folder, video_filename)
    os.makedirs(output_dir, exist_ok=True)
    
    for i, (start, end) in enumerate(scene_list):
        output_filename = os.path.join(output_dir, f"{video_filename}_scene_{i+1}.mp4")
        cmd = f"ffmpeg -i \"{video_path}\" -ss {start:.2f} -to {end:.2f} -c copy \"{output_filename}\""
        os.system(cmd)
        print(f"✅ Saved scene {i+1} to {output_filename}")
    
    print(f" Video {video_filename} successfully split into scenes at {output_dir}")

# Function to process one video at a time
def process_single_video():
    output_csv = os.path.join(output_folder, "scene_analysis.csv")
    
    if os.path.exists(input_folder):
        video_files = [f for f in os.listdir(input_folder) if f.endswith(".mp4")]
        if not video_files:
            print("❌ No video files found in the folder.")
            return
        
        print("📂 Available videos:")
        for idx, video in enumerate(video_files, start=1):
            print(f"{idx}. {video}")
        
        selected_index = int(input("Enter the number of the video you want to process: ")) - 1
        if selected_index < 0 or selected_index >= len(video_files):
            print("❌ Invalid selection.")
            return
        
        video_path = os.path.join(input_folder, video_files[selected_index])
        print(f"🔄 Processing {video_files[selected_index]}...")
        
        scene_df, scene_list = detect_scenes(video_path, output_csv)
        split_video(video_path, scene_list)
        
        print(f"✅ Finished processing {video_files[selected_index]}. Results saved to {output_csv}")
    else:
        print(f"❌ Error: Input folder {input_folder} not found!")


process_single_video()


✅ Using folder: /Users/alicja/Desktop/VideoAnalyse/Corpus/Drama
✅ Using folder: /Users/alicja/Desktop/VideoAnalyse/Corpus/SceneData/DramaData
✅ Using folder: /Users/alicja/Desktop/VideoAnalyse/Corpus/VideoSegmentation/DramaSegments
📂 Available videos:
1. Drama_04_Where Angels Fear to Tread ｜ HELENA BONHAM CARTER ｜ HELEN MIRREN ｜ Period Drama ｜  Empress Movies.mp4
2. Drama_02_PRADA presents ＂CASTELLO CAVALCANTI＂ by Wes Anderson.mp4
3. Drama_01_Nabokov on Kafka (1989).mp4
4. Drama_03_Shadowlands Anthony Hopkins Full Movie.mp4
5. Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж. Эмиль Лотяну, 1978 г.).mp4
🔄 Processing Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж. Эмиль Лотяну, 1978 г.).mp4...
📽️ Starting scene detection for /Users/alicja/Desktop/VideoAnalyse/Corpus/Drama/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж. Эмиль Лотяну, 1978 г.).mp4...
✅ Completed scene detection for /Users/alicja/Desktop/VideoAnalyse/Corpus/Drama/Drama_05_Мой ласковый 

,Start Time (s),End Time (s),Video
0,0.000000,7.132125,Drama_05_Мой ласковый и нежный зверь (FullH...
1,7.132125,14.180833,Drama_05_Мой ласковый и нежный зверь (FullH...
2,14.180833,115.156708,Drama_05_Мой ласковый и нежный зверь (FullH...
3,115.156708,139.139000,Drama_05_Мой ласковый и нежный зверь (FullH...
4,139.139000,145.562083,Drama_05_Мой ласковый и нежный зверь (FullH...
...,...,...,...
407,6335.037042,6345.756083,Drama_05_Мой ласковый и нежный зверь (FullH...
408,6345.756083,6349.176167,Drama_05_Мой ласковый и нежный зверь (FullH...
409,6349.176167,6448.442000,Drama_05_Мой ласковый и нежный зверь (FullH...
410,6448.442000,6451.528417,Drama_05_Мой ласковый и нежный зверь (FullH...


🎬 Splitting video /Users/alicja/Desktop/VideoAnalyse/Corpus/Drama/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж. Эмиль Лотяну, 1978 г.).mp4 into scenes using FFmpeg...
✅ Saved scene 1 to /Users/alicja/Desktop/VideoAnalyse/Corpus/VideoSegmentation/DramaSegments/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж_scene_1.mp4


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

✅ Saved scene 2 to /Users/alicja/Desktop/VideoAnalyse/Corpus/VideoSegmentation/DramaSegments/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж_scene_2.mp4
✅ Saved scene 3 to /Users/alicja/Desktop/VideoAnalyse/Corpus/VideoSegmentation/DramaSegments/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж_scene_3.mp4
✅ Saved scene 4 to /Users/alicja/Desktop/VideoAnalyse/Corpus/VideoSegmentation/DramaSegments/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж_scene_4.mp4


...

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

✅ Saved scene 411 to /Users/alicja/Desktop/VideoAnalyse/Corpus/VideoSegmentation/DramaSegments/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж_scene_411.mp4


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

✅ Saved scene 412 to /Users/alicja/Desktop/VideoAnalyse/Corpus/VideoSegmentation/DramaSegments/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж_scene_412.mp4
🎥 Video Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж successfully split into scenes at /Users/alicja/Desktop/VideoAnalyse/Corpus/VideoSegmentation/DramaSegments/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж
✅ Finished processing Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж. Эмиль Лотяну, 1978 г.).mp4. Results saved to /Users/alicja/Desktop/VideoAnalyse/Corpus/SceneData/DramaData/scene_analysis.csv


[out#0/mp4 @ 0x144010a00] video:8KiB audio:278KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 4.539489%
frame=  456 fps=0.0 q=-1.0 Lsize=     299KiB time=01:47:55.26 bitrate=   0.4kbits/s speed=9.65e+03x    
